In [2]:
#Lesson 9: Real-World Application : Performance of main asset classes
import yfinance as yf
import pandas as pd
import numpy as np
import math as ma

In [3]:
#Ask Whether The Risk Free Rate is Given or If It Must Be Calculated From FF3F Data

import pandas_datareader as pdr

ff3f = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench', '1926-12-31', '2024-12-31')[0]/100


while True:
    # Prompt the user and immediately handle the response
    rf_Prompt = input("Has the risk-free rate been provided already? (Yes/No): ").strip().lower()

    if rf_Prompt == "yes":
        rf_Rate = float(input("Input the given risk-free rate here: "))  # ensure input is converted to float for calculations
        print("Risk Free Rate is: ", rf_Rate)
        break
    elif rf_Prompt == "no":
        rf_Rate = ff3f['RF'].mean()  # Assuming ff3f['RF'] is already loaded and contains the relevant data
        print("The FF3F monthly risk free rate is:  ", rf_Rate)
        break
    else:
        print("Not a valid input! Please say 'Yes' or 'No'.")


C:\Users\Sean\AppData\Roaming\Python\Python311\site-packages\pandas_datareader\famafrench.py:114: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = read_csv(StringIO("Date" + src[start:]), **params)
C:\Users\Sean\AppData\Roaming\Python\Python311\site-packages\pandas_datareader\famafrench.py:114: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = read_csv(StringIO("Date" + src[start:]), **params)


Has the risk-free rate been provided already? (Yes/No):  TSLA


Not a valid input! Please say 'Yes' or 'No'.


Has the risk-free rate been provided already? (Yes/No):  No


The FF3F monthly risk free rate is:   0.002681164383561644


In [4]:

size = int(input("Enter how many stocks will be in your portfolio here: "))

portfolio = [None] * size

for i in range(size):
    portfolio[i] = input(f"Enter the ticker for stock number {i+1} here: ")
    
print("Portfolio: ", portfolio)

start = input("Start data: ") #Start Date For yfinance data pull
end = input("End date (Best to always get an extra months data for more accurate results: ") #End data for yfinance data pull

m_data = yf.download(portfolio, start = start, end = end, interval = '1mo')['Adj Close'].dropna()



m_data = m_data[portfolio]

m_data = m_data.pct_change().dropna()



m_data.head(5)

#Estimated Returns, Standard Deviations, Risk Premium

average_Data = m_data.mean()

standard_Dev = m_data.std() #Gets standard deviation from return data

risk_Premia = average_Data - rf_Rate




Enter how many stocks will be in your portfolio here:  2
Enter the ticker for stock number 1 here:  TSLA
Enter the ticker for stock number 2 here:  AAPL


Portfolio:  ['TSLA', 'AAPL']


Start data:  2010-01-01
End date (Best to always get an extra months data for more accurate results:  2020-12-31


[*********************100%%**********************]  2 of 2 completed


In [5]:
#Sigma Covariance Matrix and its inverse

df = pd.DataFrame(m_data) #Takes stocks returns and converts it into a df datafield

cov_matrix = df.cov() #Calculates the covar matrix

inverse = np.linalg.inv(cov_matrix)

print(f"Covariance Matrix: \n", cov_matrix)

Covariance Matrix: 
 Ticker      TSLA      AAPL
Ticker                    
TSLA    0.034340  0.004309
AAPL    0.004309  0.006215


In [6]:
#W_Tangeant Weights
numerator = np.dot(inverse, risk_Premia) #Numerator for w_tangent

denominator = np.dot(np.ones(len(risk_Premia)), numerator)

w_tangent = (numerator / denominator) * 100

for i in range(size):
    print(f"The W-Tan for {portfolio[i]} is {w_tangent[i].round(2)}%")

The W-Tan for TSLA is 29.53%
The W-Tan for AAPL is 70.47%


In [7]:
#Finding Risk Premium and Variance of Portfolio
w_tangent = w_tangent / 100

risk_Premia = np.array(risk_Premia)

#Portfolio Risk Premium
portfolio_RP = (risk_Premia @ w_tangent)


#Variance of Portfolio
portfolio_variance = np.dot(w_tangent.T, np.dot(cov_matrix.values, w_tangent))
print(f"The variance of the portfolio is: {portfolio_variance.round(7)}")

The variance of the portfolio is: 0.0078739


In [8]:
risk_Aversion = input("Input the risk-aversion coefficient here: ")
risk_Aversion = int(risk_Aversion)

w_OCA = portfolio_RP/ (portfolio_variance * risk_Aversion)

print(f"The weighted optimal capital allocation is: {w_OCA}")

Input the risk-aversion coefficient here:  5


The weighted optimal capital allocation is: 0.81964640851077


In [9]:
#W_oca Tangent Weights
w_tan_weights = [None] * size
rf_Alloc = (1 - w_OCA) * 100

for i in range(size):
    w_tan_weights[i] = w_OCA * w_tangent[i]
    w_tan_weights[i] = w_tan_weights[i].round(3)
    print(f" w_OCA tangent weight for {portfolio[i]} is {w_tan_weights[i]*100}%")
    
    
print(f"\nThe risk free rate allocation: {rf_Alloc.round(3)}%") #Rf-Rate Allocated (How much will we borrow)

 w_OCA tangent weight for TSLA is 24.2%
 w_OCA tangent weight for AAPL is 57.8%

The risk free rate allocation: 18.035%


In [10]:
#Sharpe Ratios for individual stocks and the portfolio

stock_Sharpes = [None] * size
for i in range(size):
    stock_Sharpes[i] = (average_Data[i] - rf_Rate) / standard_Dev[i]
    print(f" The Sharpe Ratio for {portfolio[i]}: {stock_Sharpes[i]}") #Indiv Sharpes




 The Sharpe Ratio for TSLA: 0.2914132754024235
 The Sharpe Ratio for AAPL: 0.2938171092063866


C:\Users\Sean\AppData\Local\Temp\ipykernel_12896\752195567.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stock_Sharpes[i] = (average_Data[i] - rf_Rate) / standard_Dev[i]


In [11]:
#Sharpe Ratio of Portfolio
port_Sharpe = portfolio_RP / portfolio_variance ** 0.5
print(f"The Sharpe Ratio of the portfolio: {port_Sharpe}")

The Sharpe Ratio of the portfolio: 0.3636566491933173
